<a href="https://colab.research.google.com/github/prathamsss/Relevant_Words_Analysis_NLP/blob/main/Nanos_Test_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import bs4 as bs
import urllib.request
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

In [ ]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
! echo "Installing Magnitude.... (please wait, can take a while)"
! (curl https://raw.githubusercontent.com/plasticityai/magnitude/master/install-colab.sh | /bin/bash 1>/dev/null 2>/dev/null)
! echo "Done installing Magnitude."

Installing Magnitude.... (please wait, can take a while)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   137  100   137    0     0    352      0 --:--:-- --:--:-- --:--:--   351
Done installing Magnitude.


In [ ]:
import bs4 as bs
import urllib.request
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
import pandas as pd
import spacy
from pymagnitude import MagnitudeUtils
from pymagnitude import Magnitude
from pymagnitude import Magnitude
import numpy as np
from itertools import product
import en_core_web_sm
from sklearn.metrics.pairwise import cosine_similarity
import os
class Similarity(object):
    
  def __init__(self,url,model_path):
    
    global article_text,nlp
    # try:
    scrapped_data = urllib.request.urlopen(url)#'https://github.com/plasticityai/magnitude'
    article = scrapped_data .read()

    parsed_article = bs.BeautifulSoup(article,'lxml')
    
   

    paragraphs = parsed_article.find_all('p')
    article_text = ""
    for p in paragraphs:
        article_text += p.text
        
    nlp =  en_core_web_sm.load()
       
        
  def process_text(self,text):
         
          text = text.lower()
          text=text.replace('{html}',"") 
          text = text.replace("\n","")
          cleanr = re.compile('<.*?>')
          text = re.sub(cleanr, '', text)
          text = text.strip()
          rem_url=re.sub(r'http\S+', '',text)
          text = re.sub('[0-9]+', '', rem_url)
          
          return [token.text for token in nlp(text) if not token.is_stop]
          


  def init_model(self):
                
                 model = Magnitude(model_path)
                 return model    
     
        
  def text2vec(self,doc_tok, model, dim=300):
      doc_embedding = np.zeros(dim)
      valid_words = 0
      # print((doc_tok))
      for word in doc_tok:
         
          if word in model:
             
              valid_words += 1
              doc_embedding += model.query(word)
          else:
              continue
      if valid_words > 0:
          return doc_embedding / valid_words
      else:
          return doc_embedding
 
    
      
                
  def get_docs_embedding(self,docs_tok, model, dim=300):
           all_docs_embedding = []
           for doc in docs_tok:
               all_docs_embedding.append(self.text2vec(doc, model, dim))
           return np.array(all_docs_embedding)
          
      
    
             
  def cal_embeding(self):
    my_list=[]
    my_list.append(article_text)
    mysent_dict ={}
    mysent_dict['sent'] = my_list     
    mydata = pd.DataFrame.from_dict(mysent_dict)
    mydata['processed'] = mydata['sent'].map(lambda x: S.process_text(x))
    em_model = self.init_model()
    my_headlines_embedding = self.get_docs_embedding(mydata['processed'], em_model)
    return my_headlines_embedding,mydata


  def get_relevant_words(self,search_tok, doc_tok, model):
    search_set = set()
    doc_set = set()
    word_array = set()
    for word in search_tok:
        if word in model:
            search_set.add(word)

    for word in doc_tok:
        if word in model:
            doc_set.add(word)
            
    for s in  product(search_set, doc_set):
        if model.similarity(s[0], s[1]) >= 0.45:
            word_array.add(s[1])
    return ', '.join(list(word_array))
  
  
  def semantic_search_cossim(self,search_string, docs_embeddings, em_model, data, topn=10):
    process_search_str = self.process_text(search_string)
    search_string_vect = np.array(self.text2vec(process_search_str, em_model)).reshape(1, -1)
    cosine_similarities = pd.Series(cosine_similarity(search_string_vect, docs_embeddings)[0])
    result_df = pd.DataFrame(columns=['relevant_words', 'similarity'], index=range(topn))
    
    k = 0
    
    for i, j in cosine_similarities.nlargest(int(topn)).iteritems():
        result_df['relevant_words'][k] = self.get_relevant_words(search_tok=process_search_str,
                                                            doc_tok=data.processed[i],
                                                            model=em_model)
        result_df['similarity'][k] = j
        k += 1

    return result_df

  def cal_words(self):
      search_headline = input("Products/Services : ")
      my_headlines_embedding,mydata= self.cal_embeding()
      em_model = self.init_model()
      results = self.semantic_search_cossim(search_headline, my_headlines_embedding, em_model,
                                 mydata, topn=1)


      print("\nMost important words: ",list(results['relevant_words']))
      print("\nCosine Similarities: ",list(results['similarity']))
  
      
      


model_path = '/content/drive/MyDrive/Nanos_test/crawl-300d-2M.magnitude'
url = str(input("Enter Valid URL: "))
S=Similarity(url,model_path)

S.cal_words()


# https://nanos.ai/

ModuleNotFoundError: ignored